In [ ]:
library(ggplot2)
library(RColorBrewer)
theme_set(theme_bw())
library(dplyr)

# 2018-12-13 Compare salmon and kallisto
I continue the discussion from this same day.

In [ ]:
# files and directories
tissue_ai_rootdir <- "../"
mapdir <- sprintf("%s/scratch/test_map", tissue_ai_rootdir)
kallis_fname <- sprintf("%s/kallisto_out/abundance.tsv", mapdir)
salmon_fname <- sprintf("%s/transcripts_quant/quant.sf", mapdir)

# load both files
kallis = read.csv(kallis_fname, sep='\t')
salmon = read.csv(salmon_fname, sep='\t')

If we look at the size of the two tables we realize that the number of rows does not coincide. Therefore, we first prepare an array with the names of the transcripts that the two have in common.

In [ ]:
# get names of transcripts from the two
kallis.names <- kallis$target_id
salmon.names <- salmon$Name

# intersect the names
names <- intersect(kallis.names, salmon.names)

In [ ]:
# subset the kallis table with only the names present in the intersection
kallis.common <- subset(kallis, target_id %in% names)
rownames(kallis.common) <- kallis.common$target_id
kallis.common <- kallis.common[,-1]
colnames(kallis.common) <- paste("kallisto.", colnames(kallis.common), sep="")

# and do the same for the salmon
salmon.common <- subset(salmon, Name %in% names)
rownames(salmon.common) <- salmon.common$Name
salmon.common <- salmon.common[,-1]
colnames(salmon.common) <- paste("salmon.", colnames(salmon.common), sep="")

# bind the two
X <- cbind(kallis.common, salmon.common)

Let's see whether the two algorithms agree on the transcript length.

In [ ]:
sum(X$kallisto.length == X$salmon.Length) == nrow(X)

Yes, they agree. Now let's have a look at how the tpm values compare.

In [ ]:
X

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 3)
ggplot(X, aes(x = 1+kallisto.tpm, y = 1+salmon.TPM)) + geom_point(aes(color=log(kallisto.length))) +
scale_x_continuous(trans="log10") +
scale_y_continuous(trans="log10") +
scale_colour_gradient(low = "blue", high = "red") +
labs(x = "Kallisto", y = "Salmon", title = "TPM comparison")

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 3)
ggplot(X, aes(x = 1+kallisto.est_counts, y = 1+salmon.NumReads)) + geom_point(aes(color=log(kallisto.length))) +
scale_x_continuous(trans="log10") +
scale_y_continuous(trans="log10") +
scale_colour_gradient(low = "blue", high = "red") +
labs(x = "Kallisto", y = "Salmon", title = "Counts comparison")

So there is no clear indicator of what might be the reality here. The two algorithms give different results, and it is difficult to say now which one we will choose.

I want to try to see what happens at the gene level.

In [ ]:
library(biomaRt)

In [ ]:
# load the ENSEMBL mart
mart <- useMart(biomart = "ensembl", dataset = "hsapiens_gene_ensembl")

In [ ]:
# get the gene ids corresponding to our transcripts
transcript.to.gene <- getBM(attributes = c("ensembl_transcript_id_version",
                                           "ensembl_gene_id_version"),
                  filters = "ensembl_transcript_id_version",
                  values = names,
                  mart = mart)
rownames(transcript.to.gene) <- transcript.to.gene$ensembl_transcript_id_version
X$gene_id <- transcript.to.gene$ensembl_gene_id_version

In [ ]:
# list of unique genes
genes <- unique(transcript.to.gene$ensembl_gene_id_version)

In [ ]:
# beautiful one-liner that is lightning-fast and calculates sum by gene of transcript counts
bygene <- X %>% dplyr::group_by(gene_id) %>%
dplyr::summarize(kallisto = sum(kallisto.est_counts),
                 salmon = sum(salmon.NumReads),
                 size = sum(kallisto.length))

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 3)
ggplot(bygene, aes(x = 1+kallisto, y = 1+salmon)) + geom_point(aes(color=log(size))) +
scale_x_continuous(trans="log10") +
scale_y_continuous(trans="log10") +
scale_colour_gradient(low = "blue", high = "red") +
labs(x = "Kallisto", y = "Salmon", title = "Counts comparison by gene")

Conclusion: The comparison between the two algorithms reveals that gene-level results of the two algorithms are much better.